In [73]:
#Import packages
import boto3
import json
import numpy as np
from numpy import argmax
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from urllib.parse import urlparse

#RegEx
import regex as re

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
sm_client = boto3.client('secretsmanager',region_name='us-west-2')
secret_value = sm_client.get_secret_value(SecretId = 'emr_spark_qa')
username = json.loads(secret_value['SecretString'])['emr_spark_username']
password = json.loads(secret_value['SecretString'])['emr_spark_password']
#print(username)
#print(password)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user=username,
    password=password
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
#Query some reviews
query_1 = "select distinct r.ugc_id,  mg.name, r.review_headline, r.review_comments, r.reviewer_nickname, mg.base_url, \
case when (contains(r.observations, 'URL') )  then 1 else 0 end as URL \
from analytics.review as r \
join analytics.merchant_group as mg \
on r.client_id = mg.client_id \
where contains(r.observations, 'PM') and not contains(r.observations, 'IA') and not contains(r.observations, 'IR') \
and r.created_date > '2020-01-01' \
"


#and contains(r.observations, 'URL') \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
reviews = pd.read_sql(query_1, conn)
reviews.head()
#reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID                                    NAME  \
0  283823825  OH Danish Bakery Seller Ratings          
1  283817828  San Antonio Shoes Seller Ratings         
2  283789332  Clinique                                 
3  283781663  Hershey Company                          
4  283817405  The Outdoor Gear Exchange - TERMINATED   

                                                REVIEW_HEADLINE  \
0  I have been getting and giving Kringles for many years.        
1  I have a very wide foot, and I just wish there were more ...   
2  Bon produit pour les peaux sèches                              
3  This Is Great Gum                                              
4  I will but from this site from now on!                         

                                                                                                                                                                                                                                                                    

In [78]:
reviews['ALL_TEXT'] =  reviews['REVIEW_HEADLINE'] + ' '+reviews['REVIEW_COMMENTS'] +  ' '+reviews['REVIEWER_NICKNAME']

#http://www.amazon.com -> www.amazon.com
reviews['client_URL_1'] = reviews['BASE_URL'].apply(lambda x: urlparse(x).netloc)

#Since reviewers don't always include www, look for just domain name
#www.amazon.com -> amazon.com
reviews['client_URL_2'] = reviews['client_URL_1'].apply(lambda x: '.'.join(x.split('.')[1:-1]))
#Above will return empty string '' if URL was amazon.com so fill in with random string 
#so that looking for client_URL_2 within regex_extract will return False, otherwise looking for '' within anything returns true
reviews['client_URL_2'] = reviews['client_URL_2'].replace(r'^\s*$', 'zzzzzzzzzzz', regex=True)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
#urlparse('http://www.amazon.com').netloc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
#urlparse('http://amazon.com').netloc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
reviews.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8679023 entries, 0 to 8679022
Data columns (total 10 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   UGC_ID             int64 
 1   NAME               object
 2   REVIEW_HEADLINE    object
 3   REVIEW_COMMENTS    object
 4   REVIEWER_NICKNAME  object
 5   BASE_URL           object
 6   URL                int64 
 7   ALL_TEXT           object
 8   client_URL_1       object
 9   client_URL_2       object
dtypes: int64(2), object(8)
memory usage: 662.2+ MB

In [82]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<string>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.

In [83]:
reviews.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID                                    NAME  \
0  283823825  OH Danish Bakery Seller Ratings          
1  283817828  San Antonio Shoes Seller Ratings         
2  283789332  Clinique                                 
3  283781663  Hershey Company                          
4  283817405  The Outdoor Gear Exchange - TERMINATED   

                                                REVIEW_HEADLINE  \
0  I have been getting and giving Kringles for many years.        
1  I have a very wide foot, and I just wish there were more ...   
2  Bon produit pour les peaux sèches                              
3  This Is Great Gum                                              
4  I will but from this site from now on!                         

                                                                                                                                                                                                                                                                    

In [84]:
#Ignores emails by not including @ sign before
#Include specific URL suffixes since allowing some # of characters after the period captures anywhere someone missed a space after a period
#BE is for youtu.be, GL is for goo.gl
reviews['regex_URL'] = reviews['ALL_TEXT'].str.contains(r'[^@A-Z][-A-Z0-9:%_\+~#=]+\.(CO|COM|NET|ORG|GOV|EDU|APP|IN|BE|GL|NL|LY|FR)\b', case=False)==True
#Extract actual website so that I can do a search of if it contains the client name
reviews['regex_extract'] = reviews['ALL_TEXT'].str.extract(r'([^@A-Z][-A-Z0-9:%_\+~#=]+\.(?:CO|COM|NET|ORG|GOV|EDU|APP|IN|BE|GL|NL|LY|FR)\b)', flags=re.I)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/local/lib64/python3.7/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)

In [85]:
reviews['client_URL_1'] = reviews['client_URL_1'].str.lower()
reviews['client_URL_2'] = reviews['client_URL_2'].str.lower()
reviews['regex_extract'] = reviews['regex_extract'].str.lower()

#Check if URL found by regex is client URL or not
#Return True if no URL so then in final filter look if client URL mention is True then apply URL label is False
reviews['client_URL_mention_1'] = reviews.apply(lambda x: x.client_URL_1 in x.regex_extract if x.regex_URL==True else True, axis=1)
reviews['client_URL_mention_2'] = reviews.apply(lambda x: x.client_URL_2 in x.regex_extract if x.regex_URL==True else True, axis=1)
reviews.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID                                    NAME  \
0  283823825  OH Danish Bakery Seller Ratings          
1  283817828  San Antonio Shoes Seller Ratings         
2  283789332  Clinique                                 
3  283781663  Hershey Company                          
4  283817405  The Outdoor Gear Exchange - TERMINATED   

                                                REVIEW_HEADLINE  \
0  I have been getting and giving Kringles for many years.        
1  I have a very wide foot, and I just wish there were more ...   
2  Bon produit pour les peaux sèches                              
3  This Is Great Gum                                              
4  I will but from this site from now on!                         

                                                                                                                                                                                                                                                                    

In [86]:
#Ignore mentions of "bomb.com"
#TAKES FOREVER TO RUN SINCE STRING SEARCHING
reviews['bomb_com']=reviews['regex_extract'].str.contains('bomb.com', regex=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
reviews['model_URL'] = 0

reviews.loc[(reviews['regex_URL']==True ) & ((reviews['client_URL_mention_1']==False)&(reviews['client_URL_mention_2']==False)) 
            & (reviews['bomb_com']==False) , 'model_URL'] = 1


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
FP = reviews[(reviews['URL'] == 0) & (reviews['model_URL'] == 1)]
FN = reviews[(reviews['URL'] == 1) & (reviews['model_URL'] == 0)]
TP = reviews[(reviews['URL'] == 1) & (reviews['model_URL'] == 1)]
TN = reviews[(reviews['URL'] == 0) & (reviews['model_URL'] == 0)]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
FP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1839

In [90]:
FN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1465

In [91]:
TP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3082

In [92]:
TN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8672637

In [58]:
FP.head(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          UGC_ID                             NAME  \
1273   264668227  Lowe's Microsite - TERMINATED     
5281   271310972  Dillard's                         
5388   308472173  Hammer Nutrition Seller Ratings   
8193   322463204  Fingerhut                         
12817  308842366  Cheaper Than Dirt                 
15741  296917976  SupplyHouse                       
19718  285439751  Mk Marketing, LLC                 
23183  265961342  Colgate US                        
24509  291835636  JoeBobOutfitters - Terminated     
28587  268020496  Citigroup Inc.                    

                                                    REVIEW_HEADLINE  \
1273   Great value                                                    
5281   Wardrobe update with new denim shirt                           
5388   Great experience.                                              
8193   Twice as much                                                  
12817  Cheaper than dirt still hasn't responded to my BBB com

In [59]:
from pyspark.sql.types import *

mySchema = StructType([StructField("UGC_ID", StringType(), True)\
,StructField("NAME", StringType(), True)\
,StructField("REVIEW_HEADLINE", StringType(), True)\
,StructField("REVIEW_COMMENTS", StringType(), True)\
,StructField("REVIEWER_NICKNAME", StringType(), True)\
,StructField("BASE_URL", StringType(), True)\
,StructField("URL", IntegerType(), True)\
,StructField("ALL_TEXT", StringType(), True)\
,StructField("client_URL_1", StringType(), True)\
,StructField("client_URL_2", StringType(), True)\
,StructField("regex_URL", StringType(), True)\
,StructField("regex_extract", StringType(), True)\
,StructField("client_URL_mention_1", StringType(), True)\
,StructField("client_URL_mention_2", StringType(), True)\
,StructField("bomb_com", StringType(), True)\
,StructField("model_URL", IntegerType(), True)])   



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : username,
  'sfPassword' : password,
  'sfDatabase' : 'ANALYTICS_WORKSHOP',
  'sfSchema' : 'DATA_SCIENCE',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
#Turn into spark dataframe in order to write to Snowflake
#Couldn't get write_pandas() to work to write directly from pandas to Snowflake
FP = spark.createDataFrame(FP, schema=mySchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

data is already a DataFrame
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 672, in createDataFrame
    raise TypeError("data is already a DataFrame")
TypeError: data is already a DataFrame



In [69]:
FP.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions) \
    .option("dbtable","URL_FP_test1").mode("append").save()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_BATCH_TRACKING_TABLE")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_FULL_TRACKING_TABLE")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SPARK6")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SWAP_TOKENS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.URL_FN_test1")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.CC_FP_TEST7")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…